# **Structure Output with LLM:**

In [12]:
from pydantic import BaseModel, Field, PositiveInt, ValidationError, NegativeInt
from typing import List, Optional, Union, Dict, Any, Tuple, Literal
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from datetime import datetime
from dotenv import load_dotenv
import os



load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

## **Load LLM:**

In [36]:
class LLMParameter(BaseModel):
    """Base class for LLM parameters."""
    model_name: str = Field(..., description="Name of the LLM model.")
    max_tokens: PositiveInt = Field(512, ge=1, description="Maximum number of tokens to generate.")
    temperature: Union[int, float] = Field(0.7, ge=0.0, le=1.0, description="Sampling temperature.")


params = LLMParameter(
    model_name="gpt-3.5-turbo",
    max_tokens=512,
    temperature=1
)


llm = ChatOpenAI(
    model=params.model_name,
    max_tokens=params.max_tokens,
    temperature=params.temperature
)

llm.invoke("Hi").content

'Hello! How can I assist you today?'

## **Example 01:**

In [8]:
class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: datetime | None
    tastes: dict[str, PositiveInt]


external_data1 = {
    'id': 123,
    'signup_ts': '2019-06-01 12:22',  
    'tastes': {
        'wine': 9,
        'cheese': 7,  
        'cabbage': '1',  
    },
}
external_data2 = {'id': 1, 'tastes': {}}  



try:
    # Valid data :
    print("This is ane example of validating data as expected - ")
    print(User(**external_data1) )

    #Invalid data :
    print("\n\nThis is ane example of validating data as unexpected - ")
    print(User(**external_data2) )

except ValidationError as e:
    print("\nError:")
    print(e.errors())

This is ane example of validating data as expected - 
id=123 name='John Doe' signup_ts=datetime.datetime(2019, 6, 1, 12, 22) tastes={'wine': 9, 'cheese': 7, 'cabbage': 1}


This is ane example of validating data as unexpected - 

Error:
[{'type': 'missing', 'loc': ('signup_ts',), 'msg': 'Field required', 'input': {'id': 1, 'tastes': {}}, 'url': 'https://errors.pydantic.dev/2.10/v/missing'}]


In [44]:
class Message(BaseModel):
    content: str
    role: Literal["user", "assistant", "system"]
    timestamp: datetime = Field(default_factory=datetime.now)
    
    # Validation to ensure messages aren't empty
    @classmethod
    def validate_content(cls, v):
        if not v.strip():
            raise ValueError("Message content cannot be empty")
        return v

class Conversation(BaseModel):
    messages: List[Message]
    conversation_id: str
    user_id: Optional[str] = None
    
# Example usage
conversation = Conversation(
    conversation_id="conv_12345",
    messages=[
        Message(content="Hello, how can I help you today?", role="assistant"),
        Message(content="I need information about your services", role="user")
    ]
)

conversation

Conversation(messages=[Message(content='Hello, how can I help you today?', role='assistant', timestamp=datetime.datetime(2025, 4, 16, 20, 8, 20, 353637)), Message(content='I need information about your services', role='user', timestamp=datetime.datetime(2025, 4, 16, 20, 8, 20, 353637))], conversation_id='conv_12345', user_id=None)

## **Example 02:**

In [22]:
# Step 1: Define your Pydantic model
class Response(BaseModel):
    """Response model"""
    city: str = Field(..., description="City name")
    state: str = Field(..., description="State name")
    country: str = Field(..., description="Country name")
    district: str = Field(..., description="District name")
    pincode: Union[int, None] = Field(None, description="Pincode")


# Step 2: Initialize the output parser for the model
parser = PydanticOutputParser(pydantic_object=Response)


# Step 3: Create the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that extracts location info."),
    ("user", "Extract the location info from this text:\n\n{text}\n\n{format_instructions}")
])


prompt

ChatPromptTemplate(input_variables=['format_instructions', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant that extracts location info.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'text'], input_types={}, partial_variables={}, template='Extract the location info from this text:\n\n{text}\n\n{format_instructions}'), additional_kwargs={})])

In [ ]:
# Step 4: Create the chain
chain = prompt | llm | parser

In [27]:
query = "Give me information of City Siliguri, State West Bengal, Country India, District Darjeeling, Pincode 734001."

result = chain.invoke({
    "text": query,
    "format_instructions": parser.get_format_instructions()
})

result

Response(city='Siliguri', state='West Bengal', country='India', district='Darjeeling', pincode=734001)

## **Example 03:**

In [37]:
# Step 1: Define your Pydantic model
class Response(BaseModel):
    """Response model"""
    city: str = Field(..., description="City name")
    state: str = Field(..., description="State name")
    country: str = Field(..., description="Country name")
    district: str = Field(..., description="District name")
    pincode: Union[int, None] = Field(None, description="Pincode")


# Step 2: Initialize the output parser for the model
parser = PydanticOutputParser(pydantic_object=Response)

In [38]:
# Step 3: Create the prompt template
# System Prompt: 
system_prompt = (
    "You are an CityTutor, named 'Lily' specializing in Extracting City Information."
    "This CityTutor Designed by Dibyendu Biswas, an AI/ML Engineer"
)


prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "Extract the City info from this input:\n\n{city}\n\n{format_instructions}")
])

prompt

ChatPromptTemplate(input_variables=['city', 'format_instructions'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="You are an CityTutor, named 'Lily' specializing in Extracting City Information.This CityTutor Designed by Dibyendu Biswas, an AI/ML Engineer"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['city', 'format_instructions'], input_types={}, partial_variables={}, template='Extract the City info from this input:\n\n{city}\n\n{format_instructions}'), additional_kwargs={})])

In [39]:
# Step 4: Create the chain
chain = prompt | llm | parser

In [40]:
query = "Siliguri"
response = chain.invoke(
    {
        "city": query,
        "format_instructions": parser.get_format_instructions()
    }
)
response

Response(city='Siliguri', state='West Bengal', country='India', district='Darjeeling', pincode=None)

In [41]:
query = "New Delhi"
response = chain.invoke(
    {
        "city": query,
        "format_instructions": parser.get_format_instructions()
    }
)
response

Response(city='New Delhi', state='Delhi', country='India', district='New Delhi', pincode=None)

In [42]:
query = "Kolkata"
response = chain.invoke(
    {
        "city": query,
        "format_instructions": parser.get_format_instructions()
    }
)
response

Response(city='Kolkata', state='West Bengal', country='India', district='Kolkata', pincode=None)

## **Example 04:**

In [49]:
# Step 1: Define your Pydantic model
class Response(BaseModel):
    """Response model"""
    description: str = Field(..., description="Description of the question")
    why: Union[str,None] = Field(None, description="Why it is required")
    pros: Union[List[str], str, None] = Field(None, description="Pros of the question")
    cons: Union[List[str], str, None] = Field(None, description="Cons of the question")
    examples: Union[List[str], str, None] = Field(None, description="Examples of the question")
    references: Union[List[str], str, None] = Field(None, description="References of the question")


# Step 2: Initialize the output parser for the model
parser = PydanticOutputParser(pydantic_object=Response)

In [50]:
# Step 3: Create the prompt template
# System Prompt: 
system_prompt = (
    "You are an expert AI Tutor named 'Lily', specializing in explaining all topics related to Science. "
    "This AI Tutor is designed by Dibyendu Biswas, an AI/ML Engineer. "
    "Your job is to provide clear, well-structured, and concise responses to science-related questions using the following JSON format:\n\n"
    "{format_instructions}\n\n"
    "Each response should include:\n"
    "- A short description of the topic.\n"
    "- An optional explanation of why it is important.\n"
    "- Optional lists of pros, cons, examples, and references, if applicable.\n\n"
    "Keep the explanation simple, logical, and no longer than 4 sentences in the description field. "
    "Only answer science-related questions. If the question is unclear or unrelated to science, politely ask for clarification."
    "If you don't know the answer, say 'I don't have idea about the topic.'"
)


prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "Question: {text}\n\n{format_instructions}")
])

prompt


ChatPromptTemplate(input_variables=['format_instructions', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions'], input_types={}, partial_variables={}, template="You are an expert AI Tutor named 'Lily', specializing in explaining all topics related to Science. This AI Tutor is designed by Dibyendu Biswas, an AI/ML Engineer. Your job is to provide clear, well-structured, and concise responses to science-related questions using the following JSON format:\n\n{format_instructions}\n\nEach response should include:\n- A short description of the topic.\n- An optional explanation of why it is important.\n- Optional lists of pros, cons, examples, and references, if applicable.\n\nKeep the explanation simple, logical, and no longer than 4 sentences in the description field. Only answer science-related questions. If the question is unclear or unrelated to science, politely ask for clarification.If you don

In [51]:
chain = prompt | llm | parser

In [52]:
result = chain.invoke({
    "text": "What is photosynthesis?",
    "format_instructions": parser.get_format_instructions()
})

result

Response(description='Photosynthesis is the process by which green plants, algae, and some bacteria convert light energy, usually from the sun, into chemical energy stored in glucose molecules. This process involves the absorption of carbon dioxide and the release of oxygen as a byproduct.', why='Photosynthesis is essential for life on Earth as it is the primary way in which oxygen is generated into the atmosphere. It is also the basis of the food chain, as plants are the primary producers that support all other organisms.', pros=['Essential for oxygen production which is crucial for aerobic organisms.', 'Provides the foundation of the food web, sustaining all living creatures that depend on plants for food.'], cons=['Dependent on external factors such as light availability and environmental conditions.', 'Imbalance in photosynthesis can lead to ecological disruptions and imbalances.'], examples=['An example of photosynthesis is a tree taking in carbon dioxide, water, and sunlight to p

In [54]:
display(Markdown(f"**Description:** {result.description}"))
display(Markdown(f"**Why:** {result.why}"))
display(Markdown(f"**Pros:** {result.pros}"))
display(Markdown(f"**Cons:** {result.cons}"))
display(Markdown(f"**Examples:** {result.examples}"))
display(Markdown(f"**References:** {result.references}"))

**Description:** Photosynthesis is the process by which green plants, algae, and some bacteria convert light energy, usually from the sun, into chemical energy stored in glucose molecules. This process involves the absorption of carbon dioxide and the release of oxygen as a byproduct.

**Why:** Photosynthesis is essential for life on Earth as it is the primary way in which oxygen is generated into the atmosphere. It is also the basis of the food chain, as plants are the primary producers that support all other organisms.

**Pros:** ['Essential for oxygen production which is crucial for aerobic organisms.', 'Provides the foundation of the food web, sustaining all living creatures that depend on plants for food.']

**Cons:** ['Dependent on external factors such as light availability and environmental conditions.', 'Imbalance in photosynthesis can lead to ecological disruptions and imbalances.']

**Examples:** ['An example of photosynthesis is a tree taking in carbon dioxide, water, and sunlight to produce oxygen and glucose.', "Aquatic plants undergo photosynthesis using sunlight that penetrates the water's surface."]

**References:** ['https://www.britannica.com/science/photosynthesis', 'https://www.cliffsnotes.com/study-guides/biology/biochemistry-i/photosynthesis']

## **References:**

https://github.com/dibyendubiswas1998/Agentic-AI/tree/main/LangGraph/A_LangGraph_Glossary